# Store Sales - Time Series Forecasting
赛题任务：利用2013年1月1日至2017年8月15日的厄瓜多尔地区不同商店不同商品类别不同日期的历史销售量，来预测2017年8月16日至2017年8月31日**不同商品在指定日期的销售额**。其中，时间序列预测就是利用过去一段时间的数据来预测未来一段时间内的信息。

该机器学习任务是有监督学习（有训练数据、测试数据），由于是预测不同商品在指定日期的销售额，因此该学习任务是一个回归任务（给定商品和日期，输出销售额）。

另外该任务的性能指标为均方根对数误差（Root Mean Squared Logarithmic Error，RMSLE）：

$$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$$

其中，$n$是实例的总数，$\hat{y}_i$是目标的预测值，$y_i$是目标的实际值，$\log$是自然对数（即以$e$为底）。

下面开始对数据进行分析：

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sales_train = pd.read_csv("./data/train.csv")   # 读取训练数据
sales_train.head()
#sales_train.info()
#sales_train.describe()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


训练数据中共有3000888条数据，每条数据有6个特征：
+ id 表示商品唯一标识（int64）
+ date 表示商品的销售日期（object）
+ store_nbr 标识销售产品的商店（int64）
+ family 标识所售产品的类型（object）
+ sales 表示给定日期特定商店的产品系列的总销售额（float64）
+ onpromotion 表示在给定日期在商店促销的产品系列中的商品总数（int64）

比如数据(id=262935, date=2013/05/28, store_nbr=36, family=MEATS, sales=104.678, onpromotion=0)表示：
商品262935，在2013/05/28这一天，在商店36中，作为MEATS，销售额为104.678，而2013/05/28这一天商店促销的MEATS的商品总数为0

下面用直方图的形式来表示。

In [3]:
#sales_train.hist(bins=50, figsize=(10, 7.5))  # 50列
#plt.show()

下面进行数据清理的工作。

注意到数据中可能有些异常情况，比如缺失某些属性等等，下面就来处理这一些情况。

In [4]:
print(sales_train.count())

id             3000888
date           3000888
store_nbr      3000888
family         3000888
sales          3000888
onpromotion    3000888
dtype: int64


在这里，我们简单地测试了一下sales_train中各个特征的数量，可以看出数据都是完整的，是一种理想的情况。

--------------------------------------------------------------------------------------------------
在这里，我们添加**石油数据**，由于厄瓜多尔是一个十分依赖石油的国家，因此石油数据十分重要

In [5]:
oil_data = pd.read_csv("./data/oil.csv")
oil_data.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
oil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [7]:
oil_data.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [8]:
oil_data.count()

date          1218
dcoilwtico    1175
dtype: int64

In [9]:
 
frames = [sales_train, oil_data]
train_data = pd.concat(frames)
train_data.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico
0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,NaN
1,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,NaN
2,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,NaN
3,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,NaN
4,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,NaN


In [10]:
train_data.tail()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico
1213,NaN,2017-08-25,NaN,NaN,NaN,NaN,47.65
1214,NaN,2017-08-28,NaN,NaN,NaN,NaN,46.40
1215,NaN,2017-08-29,NaN,NaN,NaN,NaN,46.46
1216,NaN,2017-08-30,NaN,NaN,NaN,NaN,45.96
1217,NaN,2017-08-31,NaN,NaN,NaN,NaN,47.26


--------------------------------------------------------------------------------------------------

在开始训练之前，我们还要处理一下训练数据
1. 将数据中的非数值属性转换为数值属性
2. 将训练数据分为训练集和验证集
3. 从训练集和验证集中分成两部分：特征和标签


In [11]:
# 1.将数据中的非数值属性转换为数值属性
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
sales_train_date = sales_train[["date"]]
sales_train[['date']] = ordinal_encoder.fit_transform(sales_train_date)
sales_train_date = sales_train[["family"]]
sales_train[['family']] = ordinal_encoder.fit_transform(sales_train_date)

sales_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,0.0,1,0.0,0.0,0
1,1,0.0,1,1.0,0.0,0
2,2,0.0,1,2.0,0.0,0
3,3,0.0,1,3.0,0.0,0
4,4,0.0,1,4.0,0.0,0


In [12]:
'''
# 2.将训练数据分为训练集和验证集-1
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_set_indices = shuffled_indices[:test_set_size]
    train_set_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_set_indices], data.iloc[test_set_indices]

sales_train_set, sales_test_set = split_train_test(sales_train, 0.2)

print('len(sales_train_set)=', len(sales_train_set))
print('len(sales_test_set)=', len(sales_test_set))
'''

'''
# 2.将训练数据分为训练集和验证集-2
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio*(2**32)

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

sales_train_set, sales_test_set = split_train_test_by_id(sales_train, 0.2, "id")

print('len(sales_train_set)=', len(sales_train_set))
print('len(sales_test_set)=', len(sales_test_set))

# 2.将训练数据分为训练集和验证集-3
from sklearn.model_selection import train_test_split

sales_train_set, sales_test_set = train_test_split(sales_train, test_size=0.2, random_state=42)
print('len(sales_train_set)=', len(sales_train_set))
print('len(sales_test_set)=', len(sales_test_set))

'''

'\n# 2.将训练数据分为训练集和验证集-2\nfrom zlib import crc32\n\ndef test_set_check(identifier, test_ratio):\n    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio*(2**32)\n\ndef split_train_test_by_id(data, test_ratio, id_column):\n    ids = data[id_column]\n    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))\n    return data.loc[~in_test_set], data.loc[in_test_set]\n\nsales_train_set, sales_test_set = split_train_test_by_id(sales_train, 0.2, "id")\n\nprint(\'len(sales_train_set)=\', len(sales_train_set))\nprint(\'len(sales_test_set)=\', len(sales_test_set))\n\n# 2.将训练数据分为训练集和验证集-3\nfrom sklearn.model_selection import train_test_split\n\nsales_train_set, sales_test_set = train_test_split(sales_train, test_size=0.2, random_state=42)\nprint(\'len(sales_train_set)=\', len(sales_train_set))\nprint(\'len(sales_test_set)=\', len(sales_test_set))\n\n'

In [13]:
# 3.将训练数据分成特征部分和标签部分

# 针对训练集
sales_train_feature = sales_train[['id','date','store_nbr','family','onpromotion']]
sales_train_label = sales_train.loc[:,['sales']]

下面就开始训练阶段了，我们需要选择机器学习模型并展开训练了。

首先，我们选择一个简单的模型：决策树回归模型。这里需要利用的sklearn库里面的决策树回归模型库。

In [14]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(sales_train_feature, sales_train_label)

上面训练出一个决策树回归模型，下面利用测试集这个模型的性能

In [ ]:
sales_test = pd.read_csv("./data/test.csv")
sales_test_date = sales_test[['date']]
sales_test[['date']] = ordinal_encoder.fit_transform(sales_test_date)
sales_test_family = sales_test[['family']]
sales_test[['family']] = ordinal_encoder.fit_transform(sales_test_family)
sales_test

In [ ]:
sample_submission_predict = pd.read_csv("./data/sample_submission.csv")
sample_submission_predict = sample_submission_predict[['id', 'sales']]
sample_submission_predict

In [ ]:
sample_submission_predict_sales = tree_reg.predict(sales_test)
sample_submission_predict[['sales']] = sample_submission_predict_sales[:,np.newaxis]
sample_submission_predict

In [ ]:
sample_submission_predict.to_csv("./data/sample_submission.csv")